In [35]:
import pandas as pd

In [36]:
file_txt = open('wr112570.txt', 'r')
file_csv = open('dataset.csv', 'w')
for line in file_txt:
    stroka = line.replace(';', ',').replace(' ', '')
    file_csv.write(stroka)

file_csv.close()
file_txt.close()

'''Создаем файл в формате csv'''

'Создаем файл в формате csv'

In [37]:
columns = ['station', 'year', 'month', 'day', 'summary', 'min_temperature', 'avarage_temperature', 'max_temperature', 'precipitation']
df = pd.read_csv('dataset.csv', names = columns)

'''Создаем датафрейм'''


'Создаем датафрейм'

In [38]:
df_cleaned = df.drop(columns=['summary', 'min_temperature', 'precipitation'], axis=1)

df_cleaned.dropna(inplace=True)
df_cleaned['station'] = df_cleaned['station'].astype(int)

df_cleaned.to_csv('cleaned_dataset.csv',  index=False)

'''Удаляем ненужные столбцы и строки с пустями значениями, записываем в новый файл '''

'Удаляем ненужные столбцы и строки с пустями значениями, записываем в новый файл '

In [39]:
df_first = pd.read_csv('cleaned_dataset_year.csv')
df_second = pd.read_csv('stations.csv')
df_new = df_first.merge(df_second, how='left', on='station')

'''Добавляем данные о регионе метеостанаций'''

'Добавляем данные о регионе метеостанаций'

In [40]:
df_new.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True)
df_new['number of region'] = df_new['number of region'].astype(int)

In [41]:
"""Считаем климатическую норму для периода 1961-1990"""

norma = round(df_new[(df_new['year'] > 1960) & (df_new['year'] < 1991)].groupby(['day','month','station'])['max_temperature'].mean(), 1)
norma
df_norma = pd.DataFrame(norma, columns=['max_temperature']).reset_index()
df_norma.rename(columns={'max_temperature':'climatic_norm'}, inplace=True)
df_climatic_norm = df_new.merge(df_norma, how='outer', on=['day', 'month','station'])

In [42]:
'''Смотрим разницу среднесуточных температур и климатической нормы в период с 2010 по 2019 гг.'''

df_result = df_climatic_norm[(df_climatic_norm['year'] > 2009) & (df_climatic_norm['year'] < 2020)]
df_result['difference'] = round(df_climatic_norm['max_temperature'] - df_climatic_norm['climatic_norm'],1)
df_result.to_csv('anomalies.csv',  index=False)

/var/folders/pf/sj5w0_zs1fs2qhqbvvcnn2500000gn/T/ipykernel_52830/1142801032.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result['difference'] = round(df_climatic_norm['max_temperature'] - df_climatic_norm['climatic_norm'],1)


In [43]:
"""Считаем аномальные дни (превышают 5 градусов от климатической нормы) в каждом регионе"""
df_days = df_result.loc[df_result['difference'] > 5]
df_region = pd.DataFrame(df_days.groupby(['region', 'year','station']).count()['day'])

region_mean = round(df_region.groupby(['region', 'year']).mean()['day'], 2)
region_mean.to_excel('count_anomalies.xlsx',  index=True)

In [44]:
df_second

,station,name of station,number of region,region
0,20046,"Им.Э.Т.Кренкеля,ГМО",11.0,Архангельская область
1,20069,Визе,4.0,Красноярский край
2,20087,Голомянный,4.0,Красноярский край
3,20289,Русский,4.0,Красноярский край
4,20292,"Им.Е.К.Федорова,ГМО",4.0,Красноярский край
...,...,...,...,...
521,37461,"Сулак,высокогорная",82.0,Республика Дагестан
522,37470,Дербент,82.0,Республика Дагестан
523,37471,Буйнакск,82.0,Республика Дагестан
524,37472,Махачкала,82.0,Республика Дагестан
